In [1]:
import pandas as pd
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report


XGBoostError: 
XGBoost Library (libxgboost.dylib) could not be loaded.
Likely causes:
  * OpenMP runtime is not installed
    - vcomp140.dll or libgomp-1.dll for Windows
    - libomp.dylib for Mac OSX
    - libgomp.so for Linux and other UNIX-like OSes
    Mac OSX users: Run `brew install libomp` to install OpenMP runtime.

  * You are running 32-bit Python on a 64-bit OS

Error message(s): ["dlopen(/Users/gnazos/VSProjects/audio-deepfake-detection/audio-deepfake-detection/.venv/lib/python3.12/site-packages/xgboost/lib/libxgboost.dylib, 0x0006): Library not loaded: @rpath/libomp.dylib\n  Referenced from: <636BF463-1886-392D-B8B3-6011C44DCEE9> /Users/gnazos/VSProjects/audio-deepfake-detection/audio-deepfake-detection/.venv/lib/python3.12/site-packages/xgboost/lib/libxgboost.dylib\n  Reason: tried: '/opt/homebrew/opt/libomp/lib/libomp.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/opt/homebrew/opt/libomp/lib/libomp.dylib' (no such file), '/opt/homebrew/opt/libomp/lib/libomp

In [ ]:
import sys
from pathlib import Path
sys.path.insert(0, str(Path.cwd().parent))

In [ ]:
import os
from configs.config_local import DATASET_PATH

PROJECT_ROOT = Path.cwd().parent
#FOR_FEATURES_DIR = PROJECT_ROOT / "FoR_dataset" / "features"
FOR_FEATURES_DIR = os.path.join(PROJECT_ROOT, "FoR_dataset" , "features")

# train_data_path = str(FOR_FEATURES_DIR / "training_features_40_2048_512_128.parquet")
# test_data_path = str(FOR_FEATURES_DIR / "testing_features_40_2048_512_128.parquet")

train_data_path = os.path.join(FOR_FEATURES_DIR, "training_features_40_2048_512_128.parquet")
test_data_path = os.path.join(FOR_FEATURES_DIR, "testing_features_40_2048_512_128.parquet")

In [ ]:
train_df = pd.read_parquet(train_data_path)
test_df = pd.read_parquet(test_data_path)


In [ ]:
print(train_df.shape)
print(test_df.shape)
print(test_df.isna().sum())
print(train_df.isna().sum())

In [ ]:
train_df = pd.read_parquet(train_data_path)
train_df.dropna(inplace=True)
test_df = pd.read_parquet(test_data_path)
test_df.dropna(inplace=True)

X_train = train_df.drop(columns=["label", "filename"])
y_train = train_df["label"].map({"real": 0, "fake": 1})

X_test = test_df.drop(columns=["label", "filename"])
y_test = test_df["label"].map({"real": 0, "fake": 1})


In [ ]:
model = XGBClassifier(eval_metric="error")
model.fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))
print(accuracy_score(y_test, y_pred))


In [ ]:
test_itw_path = os.path.join(PROJECT_ROOT, "in-the-wild-audio-deepfake", "features", "itw_features_40_2048_512_128.parquet")
test_itw_df = pd.read_parquet(test_itw_path)
test_itw_df.dropna(inplace=True)

X_test = test_itw_df.drop(columns=["label", "filename"])
y_test = test_itw_df["label"].map({"real": 0, "fake": 1})


In [ ]:
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))
print(accuracy_score(y_test, y_pred))